In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
data = pd.read_csv("../data/final/preprocessed_full.csv", index_col=[0])

Build a "plain" time-series LSTM (takes a sequence of previous values & predicts the next):

In [3]:
data = data[data["year"] < 2022][["iso3", "year", "net_migration"]]

In [4]:
data = data.pivot(index='year', columns='iso3', values='net_migration')
data

iso3,ABW,AFG,AGO,ALB,AND,ARE,ARG,ARM,ASM,ATG,...,VEN,VGB,VIR,VNM,VUT,WSM,YEM,ZAF,ZMB,ZWE
year,,,,,,,,,,,,,,,,,,,,,
1960,0.537799,0.538444,0.526967,0.541647,0.537958,0.538769,0.542850,0.540890,0.537735,0.537710,...,0.539871,0.537777,0.538057,0.537799,0.537799,0.537681,0.533554,0.542481,0.536835,0.535755
1961,0.537658,0.539312,0.525621,0.541618,0.537961,0.538810,0.545059,0.541180,0.537776,0.537630,...,0.539302,0.537764,0.538211,0.537799,0.537799,0.537633,0.533468,0.551688,0.536720,0.535674
1962,0.537648,0.539536,0.524289,0.541584,0.537964,0.538816,0.545075,0.541489,0.537775,0.537622,...,0.538883,0.537763,0.538249,0.537799,0.537799,0.537572,0.533381,0.554652,0.536622,0.535588
1963,0.537639,0.539453,0.522999,0.541515,0.537965,0.538799,0.545385,0.541807,0.537775,0.537617,...,0.538614,0.537764,0.538255,0.537799,0.537799,0.537552,0.533289,0.555372,0.536554,0.535480
1964,0.537630,0.539552,0.519986,0.541417,0.537966,0.538770,0.545802,0.542114,0.537775,0.537664,...,0.538396,0.537769,0.538257,0.535322,0.537799,0.537556,0.533198,0.556224,0.536508,0.535307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,0.537853,0.526140,0.557045,0.535381,0.538019,0.534607,0.539019,0.531281,0.537390,0.537799,...,0.335331,0.537869,0.537683,0.536847,0.537699,0.537100,0.527977,0.535155,0.542817,0.522964
2018,0.537890,0.526112,0.553058,0.535545,0.538092,0.535674,0.538960,0.531403,0.537390,0.537799,...,0.201883,0.537854,0.537686,0.537037,0.537699,0.537121,0.527970,0.544550,0.542260,0.522964
2019,0.537901,0.535798,0.552403,0.535598,0.538106,0.536693,0.538908,0.531520,0.537391,0.537799,...,0.296040,0.537862,0.537686,0.536802,0.537699,0.537188,0.527953,0.543426,0.542260,0.522964


Building sequences. Model input will be a sequence of $v$ values, and the target is the next value $v+1$:

In [49]:
def create_sequences(series: list, seq: int) -> tuple:
    sequences = []
    targets = []

    for i in range(len(series) - seq):
        sequence = series[i:i+seq]
        target = series[i+seq]

        sequences.append(sequence)
        targets.append(target)
    
    y_test = targets[-2:]
    X_test = sequences[-2:]
    y_val = targets[-4:-2]
    X_val = sequences[-4:-2]
    y_train = targets[:-4]
    X_train = sequences[:-4]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [64]:
seq = 3

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []

for country in data.columns:
    series = data[country].reset_index(drop=True)
    X_train_i, y_train_i, X_val_i, y_val_i, X_test_i, y_test_i = create_sequences(series=series, seq=seq)
    X_train.extend(X_train_i)
    y_train.extend(y_train_i)
    X_val.extend(X_val_i)
    y_val.extend(y_val_i)
    X_test.extend(X_test_i)
    y_test.extend(y_test_i)

In [83]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor([x.item() for x in y_train], dtype=torch.float32) # y_train is a list of length 1 arrays which don't work here (I hate python)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor([x.item() for x in y_val], dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor([x.item() for x in y_test], dtype=torch.float32)

Custom dataset:

In [84]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

In [89]:
train_loader = DataLoader(TimeSeriesDataset(X_train.unsqueeze(-1), y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(X_val.unsqueeze(-1), y_val), batch_size=32, shuffle=False)
test_loader = DataLoader(TimeSeriesDataset(X_test.unsqueeze(-1), y_test), batch_size=32, shuffle=False)

Example model architecture:

In [90]:

class LSTMModel(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Pass the last output through the fully connected layer
        out = self.fc(out[:, -1, :])
        return out

Parameters:

In [91]:
input_size = 1  # Since each time step in the sequence has one value
hidden_size = 256
num_layers = 2
output_size = 1  # Predicting a single value

model = LSTMModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size)
model.to(device)

LSTMModel(
  (lstm): LSTM(1, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [92]:
# Instantiate the model
model = LSTMModel(input_size, hidden_size, output_size, num_layers)
model.to(device)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for seq, target in train_loader:
        seq, target = seq.to(device), target.to(device)
        
        # Forward pass
        output = model(seq)
        loss = criterion(output, target)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


/home/kssrr/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
